In [ ]:
# Import
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from finta import TA
from talib import RSI, BBANDS

# Allow for reviewing more of the DataFrames
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read in the CSV files
googl_df = pd.read_csv("Resources/googl_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
nvda_df = pd.read_csv("Resources/nvda_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
mmm_df = pd.read_csv("Resources/mmm_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
pg_df = pd.read_csv("Resources/pg_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)

In [ ]:
# Drop unnecessary columns
column_drops =['actual_returns', 'sma_fast', 'sma_fast30', 'sma_slow', 'ema_fast', 'ema_fast30', 'ema_slow', 'std_dev', 
               'bb_upper', 'bb_lower', 'RSI', 'bb_upper_talib', 'bb_mid_talib', 'bb_lower_talib']

for df in (googl_df, nvda_df, mmm_df, pg_df):
    df.drop(columns=column_drops, inplace=True)

In [ ]:
# Create a list of individual stock DataFrame names and columns to feed loops
ticker_data = [(googl_df, "GOOGL Adj. Close"),(nvda_df, "NVDA Adj. Close"),(mmm_df, "MMM Adj. Close"),(pg_df, "PG Adj. Close")]

In [ ]:
# Loop to add actual_returns based on percentage change of Adj. Close column
for df_name, column_name in ticker_data:
    df_name["actual_returns"] = df_name[column_name].pct_change()

In [ ]:
# ticker_data_ps = [(googl_df, "GOOGL P/S (LTM)"),(nvda_df, "NVDA P/S (LTM)"),(mmm_df, "MMM P/S (LTM)"),(pg_df, "PG P/S (LTM)")]
# ticker_data_pfcf = [(googl_df, "GOOGL P/FCF (LTM)"),(nvda_df, "NVDA P/FCF (LTM)"),(mmm_df, "MMM P/FCF (LTM)"),(pg_df, "PG P/FCF (LTM)")]
# ticker_data_pe = [(googl_df, "GOOGL P/E (LTM)"),(nvda_df, "NVDA P/E (LTM)"),(mmm_df, "MMM P/E (LTM)"),(pg_df, "PG P/E (LTM)")]

# # Loop to Generate SMA and EMA columns for P/S
# for df_name, column_name in ticker_data_ps:
#     df_name["ps_sma_fast"] = df_name[column_name].rolling(window=10).mean()
#     df_name["ps_sma_slow"] = df_name[column_name].rolling(window=100).mean()
#     df_name["ps_ema_fast"] = df_name[column_name].ewm(span=10, adjust=False).mean()
#     df_name["ps_ema_slow"] = df_name[column_name].ewm(span=100, adjust=False).mean()

# # Loop to Generate SMA and EMA columns for P/FCF
# for df_name, column_name in ticker_data_pfcf:
#     df_name["pfcf_sma_fast"] = df_name[column_name].rolling(window=10).mean()
#     df_name["pfcf_sma_slow"] = df_name[column_name].rolling(window=100).mean()
#     df_name["pfcf_ema_fast"] = df_name[column_name].ewm(span=10, adjust=False).mean()
#     df_name["pfcf_ema_slow"] = df_name[column_name].ewm(span=100, adjust=False).mean()

# # Loop to Generate SMA and EMA columns for P/E
# for df_name, column_name in ticker_data_pe:
#     df_name["pe_sma_fast"] = df_name[column_name].rolling(window=10).mean()
#     df_name["pe_sma_slow"] = df_name[column_name].rolling(window=100).mean()
#     df_name["pe_ema_fast"] = df_name[column_name].ewm(span=10, adjust=False).mean()
#     df_name["pe_ema_slow"] = df_name[column_name].ewm(span=100, adjust=False).mean()

In [ ]:
ticker_data_all = [
    (googl_df, [("GOOGL P/S (LTM)", "ps"), ("GOOGL P/FCF (LTM)", "pfcf"), ("GOOGL P/E (LTM)", "pe")]),
    (nvda_df, [("NVDA P/S (LTM)", "ps"), ("NVDA P/FCF (LTM)", "pfcf"), ("NVDA P/E (LTM)", "pe")]),
    (mmm_df, [("MMM P/S (LTM)", "ps"), ("MMM P/FCF (LTM)", "pfcf"), ("MMM P/E (LTM)", "pe")]),
    (pg_df, [("PG P/S (LTM)", "ps"), ("PG P/FCF (LTM)", "pfcf"), ("PG P/E (LTM)", "pe")]),
]

for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        # Generate SMA and EMA columns for each metric (all ratios)
        df[f"{prefix}_sma_fast"] = df[column_name].rolling(window=10).mean()
        df[f"{prefix}_sma_fast30"] = df[column_name].rolling(window=30).mean()
        df[f"{prefix}_sma_slow"] = df[column_name].rolling(window=100).mean()
        df[f"{prefix}_ema_fast"] = df[column_name].ewm(span=10, adjust=False).mean()
        df[f"{prefix}_ema_fast30"] = df[column_name].ewm(span=30, adjust=False).mean()
        df[f"{prefix}_ema_slow"] = df[column_name].ewm(span=100, adjust=False).mean()

In [ ]:
# # Loop to Generate Bollinger Bands columns (manual generation)
# for df_name, column_name in ticker_data:
#     df_name["std_dev"] = df_name[column_name].rolling(window=20).std()
#     df_name["bb_upper"] = df_name["sma_fast"] + (df_name["std_dev"] * 2)
#     df_name["bb_lower"] = df_name["sma_fast"] - (df_name["std_dev"] * 2)

In [ ]:
# Loop to Generate Bollinger Bands and RSI columns off of ratios data (P/S, P/FCF, and P/E) and 
# write to new columns in respective DataFrames
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        up, mid, low = BBANDS(df[column_name], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
        df[f"{prefix}_RSI"] = RSI(df[column_name].values, timeperiod=14)
        df[f"{prefix}_bb_upper"] = up
        df[f"{prefix}_bb_mid"] = mid
        df[f"{prefix}_bb_lower"] = low

#### Storing individual stock DataFrames with SMA, EMA, Bollinger Bands/RSI to Resources folder

In [ ]:
# Write individual stock DFs w/ SMA, EMA, and Bollinger Bands/RSI generated with all ratios data to CSV files
googl_df.to_csv('Resources/googl_ratio_indicators.csv', index=True)
nvda_df.to_csv('Resources/nvda_ratio_indicators.csv', index=True)
mmm_df.to_csv('Resources/mmm_ratio_indicators.csv', index=True)
pg_df.to_csv('Resources/pg_ratio_indicators.csv', index=True)